In [ ]:
!pwd

In [ ]:
! nvcc --version

In [ ]:
! nvidia-smi

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

Install transformers

In [ ]:
# !pip install git+https://github.com/huggingface/transformers

Install the other required libraries

In [ ]:
! pip install --upgrade transformers
! pip install transformers[sentencepiece]
! pip install transformers[torch]
! pip install seqeval
! pip install conllu
! pip install wandb

Name the WandB Project

In [ ]:
# login to wandb
import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT=RTB-NER-Transfer-Learning
%env WANDB_TAGS = ["Transfer Learning", "BERT", "train"]

Run the training

In [ ]:
# ! python scripts/preprocess.py data/ciat_ner_diseases-output-iob-tags-30-train.txt bert-base-multilingual-cased 256 > data/train.txt
# ! python scripts/preprocess.py data/ciat_ner_diseases-output-iob-tags-30-test.txt bert-base-multilingual-cased 256 > data/test.txt
# ! python scripts/preprocess.py data/ciat_ner_diseases-output-iob-tags-30-validate.txt bert-base-multilingual-cased 256 > data/dev.txt

In [ ]:
!python run_ner.py ./configs/train_config_bert_30.json